In [1]:
# import data and functions module
from functions import *
path_308 = 'data/308.15.txt'
path_358 = 'data/358.15.txt'
path_408 = 'data/408.15.txt'

data_308 = pd.read_csv(path_308, delimiter='\t')
data_358 = pd.read_csv(path_358, delimiter='\t')
data_408 = pd.read_csv(path_408, delimiter='\t')

Q2. Table 1. molar volume, compressibility factor for CO 2 at 35o C, 85o C and 135o C over 0.18-18 MPa


In [ ]:
# Define pressure and temperature values as NumPy arrays
P_vals = 1000000*np.array([0.18, 2, 4, 6, 8, 10, 12, 14, 16, 18]) # Pa
T_vals = np.array([308.15, 358.15, 408.15]) # K
datasets = [data_308, data_358, data_408]

table1 = pd.DataFrame(columns=["P (MPa)", "v(m3/mol)", "Z", "PR-Z", "% Diff Z"])
i = 0 # index for datasets
j = 0 # index for NIST data
k = 0 # index for table1

for T in T_vals:
  for P in P_vals:
    table1.at[k, "P (MPa)"] = P/1000000

    PRZ = calc_PRZ(T, P) # Calculate PR-Z using Peng-Robinson EOS
    table1.at[k, "PR-Z"] = PRZ

    data = datasets[i]
    Z = calc_Z(T, P, data["Volume (m3/mol)"][j]) # Calculate Z using NIST data
    table1.at[k, "v(m3/mol)"] = data["Volume (m3/mol)"][j]
    table1.at[k, "Z"] = Z

    percent_diffz = percent_diff(Z,PRZ) # Calculate percent difference between PR-Z and NIST Z
    table1.at[k, "% Diff Z"] = percent_diffz

    if j < 9:
      j += 1
    k += 1  
  j = 0
  i += 1

Q2. Table 2. Enthalpy difference for CO2 at specified P and T

In [3]:
P2_vals = 1000000*np.array([2, 8, 16])
table2 = pd.DataFrame(columns=["P (MPa)", "Delta H T85-T35 (kJ/mol)", "Delta Hdep T85-T35 (kJ/mol)", "Delta H T135-T35 (kJ/mol)", 
                               "Delta Hdep T135-T35 (kJ/mol)"])

l=0 # index for table2
m = [1, 4, 8] # index in data, starting at P=2 MPa

for P in P2_vals: 
    table2.at[l, "P (MPa)"] = P/1000000
    DeltaH1 = data_358["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    table2.at[l, "Delta H T85-T35 (kJ/mol)"] = DeltaH1

    DeltaHid1 = calc_idealH(308.15, 358.15)
    Hres308 = res_enth(308.15, P, calc_PRZ(308.15, P))
    Hres358 = res_enth(358.15, P, calc_PRZ(358.15, P))
    DeltaHdep1 = DeltaHid1 + (Hres358 - Hres308)
    table2.at[l, "Delta Hdep T85-T35 (kJ/mol)"] = DeltaHdep1

    DeltaH2 = data_408["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    table2.at[l, "Delta H T135-T35 (kJ/mol)"] = DeltaH2

    DeltaHid2 = calc_idealH(308.15, 408.15)
    Hres408 = res_enth(408.15, P, calc_PRZ(408.15, P))
    DeltaHdep2 = DeltaHid2 + (Hres408 - Hres308)
    table2.at[l, "Delta Hdep T135-T35 (kJ/mol)"] = DeltaHdep2

    l += 1
table2

,P (MPa),Delta H T85-T35 (kJ/mol),Delta Hdep T85-T35 (kJ/mol),Delta H T135-T35 (kJ/mol),Delta Hdep T135-T35 (kJ/mol)
0,2.0,2.182,2.187044,4.359,4.431875
1,8.0,6.46,5.933629,9.32,8.850344
2,16.0,6.441,6.390265,10.909,10.696158


Q5. Table 3: Welec done at each stage of compression for CO2
The compressor is assumed to be:
- adiabatic
- neglibible change in kinetic and potential energy
- continuous flow at steady state
- no reactions

In [9]:
inlet_P = [1.8, 5.4, 16.2] # MPa
outlet_P = [5.4, 16.2, 18]  # MPa
outlet_T = [358.15, 408.15, 408.15] # K
H_in = [308.15, 308.15, 308.15] # enthalpy from NIST
H_s = [6.5, 6.5, 6.5] # entropy from NIST
table3 = pd.DataFrame(columns=["Inlet P (MPa)", "Outlet P (MPa)", "Inlet T (oC)", "Outlet T (oC)","Welec (MW)"])
for i in range(3):
    table3.at[i, "Inlet P (MPa)"] = inlet_P[i]
    table3.at[i, "Outlet P (MPa)"] = outlet_P[i]
    table3.at[i, "Inlet T (oC)"] = 40
    table3.at[i, "Outlet T (oC)"] = outlet_T[i] - 273.15
    table3.at[i, "Welec (MW)"] = calc_work(H_in[i], Hreal_out(H_in[i], H_s[i]))

NameError: name 'Hreal_out' is not defined

Q6. Table 4: Heat duty of inter-stage coolers using dep function expression and ideal gas heat capacity (Cp)
The coolers are assumed to be:
- not have any moving parts
- neglibible change in kinetic and potential energy
- continuous flow at steady state
- no reactions

In [4]:
const_P = [5.4, 16.2, 18] # MPa
inlet_T = [412.24, 358.15, 408.15] # K
t = 0 # index for inlet_T
table4 = pd.DataFrame(columns=["Inlet P (MPa)", "Outlet P (MPa)", "Inlet T (oC)", "Outlet T (oC)", "Q (MW)"])
for P in const_P:

    table4.at[t, "Inlet P (MPa)"] = P
    table4.at[t, "Outlet P (MPa)"] = P
    table4.at[t, "Inlet T (oC)"] = inlet_T[t] - 273.15
    table4.at[t, "Outlet T (oC)"] = 40
    Q = calc_Q(inlet_T[t], 40+273.15, P*1000000, calc_mol_flow())
    table4.at[t, "Q (MW)"] = Q
    t += 1